In [10]:
import requests
import json
import time
from kafka import KafkaProducer
from datetime import datetime 
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [12]:
print("Connecting to Kafka....")

producer = KafkaProducer(
    bootstrap_servers=['kafka:29092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

print("Connected to Kafka")

Connecting to Kafka....
Connected to Kafka


In [13]:
def get_authorization_token(client_id:str, client_secret: str):
    
    url = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"
    
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    response = requests.post(url, data=data)

    if response.status_code == 200:

        return response.json()["access_token"]
    else:
        
        raise Exception(f"Ошибка: {response.status_code}, {response.text}")
    

In [15]:
def get_opensky_data(token):
    try:
        headers = {"Authorization": f"Bearer {token}"}
        
        url = "https://api.opensky-network.org/api/states/all"
        params = {
            'lamin': 35.0,   # минимальная широта
            'lamax': 70.0,   # максимальная широта
            'lomin': -10.0,  # минимальная долгота
            'lomax': 40.0    # максимальная долгота
        }
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()

        data = response.json()
        states = data.get('states',[])

        print(f'RECEIVED: {len(states)}')
        
        return states

    except Exception as e:
        print(e)

        return []

    

In [17]:
print("\n" + "="*50)
flights = get_opensky_data(get_authorization_token(CLIENT_ID, CLIENT_SECRET))
try:
    while True:
        for flight in flights[:5]:
            if flight:
                message = {
                    "icao24": flight[0],
                    "callsign": flight[1],
                    "origin_country": flight[2],
                    "time_position": flight[3],
                    "last_contact": flight[4],
                    "longitude": flight[5],
                    "latitude": flight[6],
                    "baro_altitude": flight[7],
                    "on_ground": flight[8],
                    "velocity": flight[9],
                    "true_track": flight[10],
                    "vertical_rate": flight[11],
                    "sensors": flight[12],
                    "geo_altitude": flight[13],
                    "squawk": flight[14],
                    "spi": flight[15],
                    "position_source": flight[16],
                    # "category": flight[17]
                }
        
                producer.send('opensky', value=message)
                print(f"SEND: {message['callsign']} ({message['icao24']})")
        
        time.sleep(10)

except KeyboardInterrupt:
    print('STOPPING....')
finally:
    producer.close()
    print('STOPPED')
                
    


RECEIVED: 2460
SEND: SWR248Z  (4b1819)
SEND: TVF26EN  (39de4d)
SEND: TVF80PL  (39de4c)
SEND: CMB186   (a57b27)
SEND: SWR791T  (4b1800)
SEND: SWR248Z  (4b1819)
SEND: TVF26EN  (39de4d)
SEND: TVF80PL  (39de4c)
SEND: CMB186   (a57b27)
SEND: SWR791T  (4b1800)
SEND: SWR248Z  (4b1819)
SEND: TVF26EN  (39de4d)
SEND: TVF80PL  (39de4c)
SEND: CMB186   (a57b27)
SEND: SWR791T  (4b1800)
SEND: SWR248Z  (4b1819)
SEND: TVF26EN  (39de4d)
SEND: TVF80PL  (39de4c)
SEND: CMB186   (a57b27)
SEND: SWR791T  (4b1800)
SEND: SWR248Z  (4b1819)
SEND: TVF26EN  (39de4d)
SEND: TVF80PL  (39de4c)
SEND: CMB186   (a57b27)
SEND: SWR791T  (4b1800)
STOPPING....
STOPPED
